In [228]:
import yfinance as yf
import tensorflow as tf
import pandas as pd
import ta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from keras._tf_keras.keras.models import Sequential
from keras._tf_keras.keras.layers import LSTM, Dense, Dropout
from keras._tf_keras.keras.regularizers import L2

In [229]:
stock_chosen = yf.Ticker("AAPL")

stock_data = stock_chosen.history(start='2010-01-01', end='2024-09-01', interval="1d")

In [230]:
stock_data['MA_20'] = ta.trend.sma_indicator(stock_data['Close'], window=20)
stock_data['MA_50'] = ta.trend.sma_indicator(stock_data['Close'], window=50)

In [231]:
stock_data['RSI'] = ta.momentum.rsi(stock_data['Close'], window=14)

In [232]:
macd = ta.trend.MACD(stock_data['Close'])
stock_data['MACD'] = macd.macd()
stock_data['MACD_Signal'] = macd.macd_signal()
stock_data['MACD_Diff'] = macd.macd_diff()

In [233]:
bollinger = ta.volatility.BollingerBands(stock_data['Close'], window=20, window_dev=2)
stock_data['Bollinger_High'] = bollinger.bollinger_hband()
stock_data['Bollinger_Low'] = bollinger.bollinger_lband()

In [234]:
stock_data.dropna(inplace=True)

In [235]:
sp500 = yf.download('^GSPC', start='2010-01-01', end='2024-09-01', progress=False)
sp500 = sp500['Close'].rename('S&P500_Close')

In [236]:
stock_data.index = stock_data.index.tz_localize(None)
sp500.index = sp500.index.tz_localize(None)

In [238]:
stock_data = stock_data.join(sp500, how='inner')


In [239]:
print(stock_data.columns)


After trial and error, I've found that the features I want to test against are better when kept simple.

In [240]:
# relevant features
features = ['Open', 'Close', 'Volume', 'S&P500_Close']
stock_data = stock_data[features].dropna()


In [241]:
stock_data['Target'] = stock_data['Close'].shift(-1)
stock_data.dropna(inplace=True)

In [242]:

X = stock_data[features]
y = stock_data['Target']

In [243]:
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.15, shuffle=False)  # Shuffle=False for time series


In [244]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.1765, shuffle=False)

Implement a MinMaxScaler to ensure that all features contribute equally to the learning process

In [245]:
scaler = MinMaxScaler(feature_range=(0, 1))

In [246]:
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

LSTM Neural Network:

In [247]:
window_size = 120

def create_sequences(X, y, window_size):
    Xs, ys = [], []
    for i in range(len(X) - window_size):
        Xs.append(X[i:i+window_size])
        ys.append(y[i+window_size])
    return np.array(Xs), np.array(ys)

In [248]:
target_scaler = MinMaxScaler(feature_range=(0, 1))
y_train_scaled = target_scaler.fit_transform(y_train.values.reshape(-1, 1))  # Reshape required for scaling
y_val_scaled = target_scaler.transform(y_val.values.reshape(-1, 1))  # Transform validation target
y_test_scaled = target_scaler.transform(y_test.values.reshape(-1, 1))  # Transform test target


In [249]:
X_val_lstm, y_val_lstm = create_sequences(X_val_scaled, y_val.values, window_size)
X_train_lstm, y_train_lstm = create_sequences(X_train_scaled, y_train.values, window_size)
X_test_lstm, y_test_lstm = create_sequences(X_test_scaled, y_test.values, window_size)

In [250]:
model = Sequential()
model.add(LSTM(100, return_sequences=True, kernel_regularizer=L2(0.001), input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
model.add(Dropout(0.3))
model.add(LSTM(100, return_sequences=False, kernel_regularizer=L2(0.001)))
model.add(Dropout(0.3))
model.add(Dense(50, kernel_regularizer=L2(0.001)))
model.add(Dense(1))

In [251]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [252]:
history = model.fit(
    X_train_lstm, y_train_lstm,
    epochs=50,
    batch_size=32,
    validation_stock_data=(X_val_lstm, y_val_lstm),  # Use validation set
    verbose=1
)

In [253]:
# predict
y_pred_lstm = model.predict(X_test_lstm)
y_pred_lstm = y_pred_lstm.flatten()

# evaluate error
mse_lstm = mean_squared_error(y_test_lstm, y_pred_lstm)
rmse = np.sqrt(mse_lstm)
print(f"Root Mean Squared Error (RMSE): {rmse}")

average_actual_price = np.mean(y_test_lstm)

rmse_percentage_error = (rmse / average_actual_price) * 100
print(f"RMSE Percentage Error: {rmse_percentage_error:.2f}%")


In [254]:

plt.figure(figsize=(14, 7))

train_size = len(y_train_lstm)
test_size = len(y_test_lstm)

plt.plot(range(train_size), y_train_lstm, label='Actual Training Prices', color='blue')
plt.plot(range(train_size, train_size + test_size), y_test_lstm, label='Actual Test Prices', color='cyan')
plt.plot(range(train_size, train_size + test_size), y_pred_lstm, label='Predicted Test Prices', color='green')

plt.title('LSTM: Actual vs Predicted Prices (Train and Predict)')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()

plt.show()